# XGB Models

The Data will be split by week and use week 9 as a validation set 

The Workflow will be as follow: 

    ** Use the default xgbregresor to train and plot feature importance
    ** Use a gridsearch to find the ideal hyperparameters (with only 50 estimators) 
    ** Train using all data and 500 estimators as our final model 
    

## SetUp Variables

In [1]:
predict_w11 = False #I'm going to use this variable to know when to use w10 for training
dataset_file = 'jorge_dataset_modifiedwPred.csv'

## Imports 

In [2]:
import os

mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'

os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

import xgboost as xgb

C:\Anaconda\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [3]:
import pandas as pd
import numpy as np
import re 
import os
import time
from IPython.display import display
import matplotlib.pyplot as plt
from sklearn.cross_validation import LabelKFold
from sklearn.grid_search import GridSearchCV
import seaborn as sns
%matplotlib inline

## Util functions

In [4]:
# Load computed Dataset
def load_dataset(folder = ''):
    
    
    filename = dataset_file
    filepath = os.path.join(folder,filename)        
    
    df_train = pd.read_csv(filepath,                           
                           dtype  = {'Producto_ID':'int32',
                                     'Semana':'int8',
                                     'Cliente_ID':'int32',
                                     'Agencia_ID':'uint16',
                                     'Canal_ID':'int8',
                                     'Ruta_SAK':'int32',
                                     'ZipCode':'int16',                                     
                                     'Mean2':'float32','Mean3':'float32','Mean4':'float32',
                                     'Mean6':'float32','Mean7':'float32',
                                     'Median2':'float32','Median3':'float32','Median4':'float32',
                                     'Median6':'float32','Median7':'float32',
                                     'Last_per_Ruta_SAK':'float32','Last_per_Cliente_ID':'float32',
                                     'Demanda_uni_equil':'float32'})
    return df_train

## Load Dataset 

In [5]:
df = load_dataset()

In [6]:
print df.columns
display(df.head())
print df.dtypes

Index([u'Unnamed: 0', u'Agencia_ID', u'Canal_ID', u'Cliente_ID',
       u'Demanda_uni_equil', u'Producto_ID', u'Ruta_SAK', u'Semana',
       u'ZipCode', u'id', u'Last_per_Cliente_ID', u'Last_per_Ruta_SAK',
       u'week_ct', u'Log_Target_mean_lag1', u'Log_Target_mean_lag2',
       u'Log_Target_mean_lag3', u'Log_Target_mean_lag4', u'Lags_sum', u'brand',
       u'Qty_Ruta_SAK_Bin', u'num_prod', u'num_prod_uni'],
      dtype='object')


,Unnamed: 0,Agencia_ID,Canal_ID,Cliente_ID,Demanda_uni_equil,Producto_ID,Ruta_SAK,Semana,ZipCode,id,...,week_ct,Log_Target_mean_lag1,Log_Target_mean_lag2,Log_Target_mean_lag3,Log_Target_mean_lag4,Lags_sum,brand,Qty_Ruta_SAK_Bin,num_prod,num_prod_uni
0,0,1110,7,15766,1.386294,1212,3301,3,2008,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,23.0,23.0
1,1,1110,7,15766,1.609438,1216,3301,3,2008,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,23.0,23.0
2,2,1110,7,15766,1.609438,1238,3301,3,2008,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,23.0,23.0
3,3,1110,7,15766,1.609438,1240,3301,3,2008,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,23.0,23.0
4,4,1110,7,15766,1.386294,1242,3301,3,2008,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,23.0,23.0


Unnamed: 0                int64
Agencia_ID               uint16
Canal_ID                   int8
Cliente_ID                int32
Demanda_uni_equil       float32
Producto_ID               int32
Ruta_SAK                  int32
Semana                     int8
ZipCode                   int16
id                      float64
Last_per_Cliente_ID     float32
Last_per_Ruta_SAK       float32
week_ct                 float64
Log_Target_mean_lag1    float64
Log_Target_mean_lag2    float64
Log_Target_mean_lag3    float64
Log_Target_mean_lag4    float64
Lags_sum                float64
brand                   float64
Qty_Ruta_SAK_Bin        float64
num_prod                float64
num_prod_uni            float64
dtype: object



#### Since I use Last -> Discard Week 3

#### Test data is week 10 and 11

In [7]:
validation = 9 

if predict_w11:
    validation = 10

df.Demanda_uni_equil[df.Demanda_uni_equil < 0] = 0
df_validation = df[df.Semana == validation]
df_train = df[(df.Semana != 3) & (df.Semana < validation)]
df_test = df[df.Semana > validation]
del df

C:\Anaconda\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
# Remove Columns that are not features 
#drop_columns = ['id','Semana','Demanda_uni_equil','DemandaAgencia_ID','DemandaCanal_ID','DemandaRuta_SAK','DemandaZipCode']
drop_columns = ['id','Semana','Demanda_uni_equil']

model_002 = ['Mean3','Median3', 'Mean2', 'Median2','Mean4','Median4', 'Mean6', 'Median6','Mean7','Median7']
model_003 = ['ZipCode','Cliente_ID','Agencia_ID','Ruta_SAK','Producto_ID','Canal_ID','DemandaRuta_SAK']


In [9]:
Semana = df_train.Semana

features = df_train.drop(drop_columns,axis=1).columns


# Feature Importance 

### Lets train a default model and see the feature importance plot

In [10]:
gbm = xgb.XGBRegressor(seed=1234)
#%time gbm.fit(X_train, Y_train, eval_metric='rmse', verbose=True)

In [11]:
#plt.figure(figsize=(10, 10))
#xgb.plot_importance(gbm.booster())

## Parameter Tuning (YaY!) 

#### Cross-Validation and GridSearch
     Since I'm using xgboost I'm going to use the sklearn API so I can use GridSearchCV
          
http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html

    Also, Since this is not just normal data where each row is a data point but each point correspond to a point in a sequence (Weekly delivers). I'm going to use the 'Semana' to create the folds, for this refer to sklearn Label KFold 

http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.LabelKFold.html#sklearn.cross_validation.LabelKFold

### GridSearch 

    Pretty common grid 
    
        * Max depth of the tree
        * n estimators: # Trees
        * Learning Rate
        * subsample
        * alpha and lambda regs
        
please refer:
http://xgboost.readthedocs.io/en/latest/python/python_api.html

In [12]:
param = {'max_depth':[10,6], 
        'n_estimators':[50] ,
        'learning_rate':[0.1,0.05],         
         'subsample': [0.5,1],
         #'reg_alpha':[0,1], #L2 term
         #'reg_lambda':[0,1]#, #L1 tem            
         #'silent': [False]
        }

best_param = {'max_depth':[10], 
        'n_estimators':[100] ,
        'learning_rate':[0.1],         
         'subsample': [0.5],
         #'reg_alpha':[0,1], #L2 term
         #'reg_lambda':[0,1]#, #L1 tem            
         #'silent': [False]
        }

I tried the gridsearch with CV, takes too long in my machine (4Cores 28GB) I gave up at 12hrs running.

# GO! 
###### and probably get a coffe

In [13]:
grid_scores_ = pd.DataFrame(columns=['max_depth','learning_rate','n_estimators','subsample','score' ])
from sklearn.grid_search import ParameterGrid

for g in ParameterGrid(best_param):
    start = time.time()
    
    grid_gbm = xgb.XGBRegressor(**g)
    grid_gbm.fit(X_train, Y_train,eval_set=[(X_valid,Y_valid)] ,eval_metric='rmse', early_stopping_rounds=10)
    
    print "Score for this combination: ", grid_gbm.evals_result_.values()[-1].values()[-1][-1]
    g['rmse'] = grid_gbm.evals_result_.values()[-1].values()[-1][-1] #I'm sure there must be a better way to get this, but this nested dict is annoying
    grid_scores_ = grid_scores_.append(g, ignore_index=True)
            
    print("total time taken this loop: ", time.time() - start)

    
    
display(grid_scores_)    
#print "OOB: %0.5f" % best_score 
#print "Grid:", best_grid

NameError: name 'X_train' is not defined

## Retrain model with best params and test RMSLE


ReTrain the Model with all data 

In [ ]:
#X = X_train.append(X_valid, ignore_index=True)
#Y = Y_train.append(Y_valid)
df_train = df_train.append(df_validation, ignore_index=True).reset_index()

In [ ]:
df_train = df_train.reset_index()

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.grid_search import ParameterGrid

for g in ParameterGrid(best_param):
    start = time.time()
    X = df_train[features]
    Y = df_train['Demanda_uni_equil']
    gbm = xgb.XGBRegressor(**g)
    gbm.fit(X, Y ,eval_metric='rmse',early_stopping_rounds=10)
       
            
    print("total time taken this loop: ", time.time() - start)

pred = gbm.predict(X)

print "R2 = ",r2_score(np.float64(Y),np.float64(pred))
print "RMSE", np.sqrt(mean_squared_error(np.float64(Y),np.float64(pred)))

In [ ]:
# Save the Model
gbm._Booster.save_model('xgb_bimbo.model')

### Generate Submission  


For week 10 the prediction is easy, use the inputs normally from week 9 

For Week 11 the prediction is more complicated because we need to update the following features:

    -Last_per_Cliente: From week 10 (predicted)
    -Last_per_Route: From week 10 (predicted) 
    
For Last I have to run the code again    

In [ ]:
def add_Last(df ,clusters = ['Cliente_ID']):
    
    
    validclusters = np.intersect1d(clusters, df.columns)
    not_valid_clusters = np.setdiff1d(clusters, df.columns)
    
    for non in not_valid_clusters:
        
        print "Cluster ", non, " not found in the dataset"            
    
    drop_columns = ['Venta_uni_hoy','Venta_hoy','Dev_uni_proxima','Dev_proxima']
    try:
        df = df.drop(drop_columns, 1) # Try to drop the columns if the dataset has them
    except:
        df = df # no drop 
    
    for cluster in validclusters:
        
        print 'Working on cluster:', cluster                                                    
        
        target_col = 'Last_per_'+cluster
        group = [cluster,'Producto_ID','Semana']
                    
        try:
            df = df.drop(target_col,1) # If the target exist then remove it
        except:
            df = df 
        
        # First Compute the Mean by grouping the Cluster-Producto per Week Demand 
        # Then shift by Product_ID - we want to treat each products independent 
        # Lastly reset the index because we don't wanna change the dataset
        shifted_df = ( df.groupby(group).agg({'Demanda_uni_equil': np.mean})['Demanda_uni_equil'] \
                                         .groupby(level=1).shift(1) \
                                        .reset_index())
        
        shifted_df = shifted_df.fillna(0)   # Fill NaN with 0 
        shifted_df['Demanda_uni_equil'] = np.float32(shifted_df['Demanda_uni_equil']) #reset index resets the dtypes
        shifted_df.rename(columns={'Demanda_uni_equil': 'Last_per_'+cluster }, inplace=True) # rename cols        
        df = df.merge(shifted_df, on=group)
        
        #Note to self: break the line using \                               
        
    return df

In [ ]:
print df_train.dtypes
print df_test.dtypes

In [ ]:
# Concat df_test and train 
df = df_train.append(df_validation, ignore_index=True).append(df_test,ignore_index=True)

In [ ]:
#predict week 10
features = X.columns
if (~predict_w11):
    df['Demanda_uni_equil'][df.Semana == 10] = gbm.predict(df[features][df.Semana ==10])

In [ ]:
print df['Demanda_uni_equil'][df.Semana == 10].isnull().any()
display(df['Demanda_uni_equil'][df.Semana == 10].describe())


In [ ]:
def pablo_lag(df):
    semana_client_prod_mean = df.groupby(['Semana','Cliente_ID','Producto_ID'],as_index=False).agg({'Demanda_uni_equil': 'mean'})

    #here we add the number of lags we want
    lag=4
    target_col = 'Demanda_uni_equil'
    for i in range(1,lag+1):
        semana_client_prod_mean['Semana'] += 1
        semana_client_prod_mean.rename(columns={target_col: 'Log_Target_mean_lag%d' %(i)}, inplace=True)  
        
        try:
            df = df.drop('Log_Target_mean_lag%d' %(i),1) # If the target exist then remove it
        except:
            df = df 
            
        df = pd.merge(df,semana_client_prod_mean, how = 'left', on = ['Semana','Cliente_ID','Producto_ID'])
        df['Log_Target_mean_lag%d' %(i)].fillna(0, inplace=True) # we replace the client-product log mean NaN/Not found on the week before with ZERO
        target_col = 'Log_Target_mean_lag%d' %(i)
        #data = data[data.Semana != i+2] # here we delete the week rows we dont have lags for
        
    return df
def pablo_lag_sum(df):
        #We want to sum the lags up until week 9, this means that we need to sum lag2 and up.
    df['Lags_sum'] = 0
    for i in range(1,lag+1):
        df['Lags_sum'] += df['Log_Target_mean_lag%d' %(i)]
        
    return df

def Lu_unique_product(df):
        # for every client, get the number of products ordered in the current week
    df_num_prods = df.groupby(['Semana','Cliente_ID'])['Producto_ID'].apply(lambda x: len(x)).reset_index()
    df_num_prods.rename(columns={df_num_prods.columns[-1]: 'num_prod'}, inplace=True)


    # for every client, get the number of unique products ordered in the current week
    df_num_prods_unique = df.groupby(['Semana','Cliente_ID'])['Producto_ID'].apply(lambda x: len(x.unique())).reset_index()
    df_num_prods_unique.rename(columns={df_num_prods_unique.columns[-1]: 'num_prod_uni'}, inplace=True)

    # merge to the dataset
    df = df.merge(df_num_prods,on=['Semana','Cliente_ID'])
    df = df.merge(df_num_prods_unique, on=['Semana','Cliente_ID'])
    
    return df

In [ ]:
#df =df.drop(model_002,1)
# Re Run Last Code
df = add_Last(df,['Cliente_ID','Ruta_SAK'])

# Re Run Lagged Code 
lag=4
df = pablo_lag(df)

# Re Run Lag Sum Code
df = pablo_lag_sum(df)

# Re Run Unique Product 
df = Lu_unique_product(df)

print df.dtypes

## Do I Re Train with week 10 Predictions to make a more accurate week 11 predictions? 

### TODO

Do I have weeks with demand = 0 ? 



In [ ]:
# Predict week 11
df['Demanda_uni_equil'][df.Semana == 11] = gbm.predict(df[features][df.Semana ==11])

In [ ]:
# take only the last two weeks
df_submit = df[df.Semana >9]
# Revert the log1p to the demand  
df_submit.Demanda_uni_equil[df_submit.Demanda_uni_equil < 0] = 0 # Set Negative Values to 0
df_submit.Demanda_uni_equil = df_submit.Demanda_uni_equil.apply(lambda x: np.expm1(x))

In [ ]:
display(df_submit[['id','Demanda_uni_equil']][df_submit.Semana==10].describe())
display(df_submit[['id','Demanda_uni_equil']][df_submit.Semana==11].describe())

In [ ]:
# How many are negative ? 

df_submit[(df_submit.Semana == 10) & (df_submit.Demanda_uni_equil <= 0)].describe()

In [ ]:
# Save csv with sumission data 
df_submit[['id','Demanda_uni_equil']].to_csv('submit_model_j006.csv',index=False)